### Chicago Census Tract Weather Data
By: Angel Moreno

**Endpoint**: https://open-meteo.com/   

**Variables:**
- Temperature (180m)
- Relative Humidity
- Precipitation
- Cloud Cover
- Visibility
- Surface Pressure
- Wind Speed (180m)
- Wind Direction (180m)


In [1]:
import pandas as pd
import geopandas as gpd
import time
import requests

In [2]:
# all tracts in Illinois as of 2020 (~3,200)
tracts = gpd.read_file("Tiger_Illinois_Tracts_2020.zip")  

# Load Chicago tracts shapefile (878 features)
chicago_tracts_raw = gpd.read_file("Chicago_Tracts_2020.zip")

# Dissolve into one boundary (the union of all Chicago tracts)
chicago_boundary = chicago_tracts_raw.dissolve()
## this is joining all 878 tracts into a single multipolygon  

# Match CRS
tracts = tracts.to_crs(chicago_boundary.crs)
## crs -> Coordinate Reference System

chicago_tracts = gpd.sjoin(tracts, chicago_boundary, how="inner", predicate="intersects")
## we keep the tracts that intersect with the previously defined chicago multipolygon

# extract the latitude and longitude needed for the api params
chicago_tracts['centroid'] = chicago_tracts.geometry.centroid
chicago_tracts['latitude'] = chicago_tracts.centroid.y
chicago_tracts['longitude'] = chicago_tracts.centroid.x

# this is the dataframe containing the ~866 Chicago tracts as of 2020 census data
chicago_tracts

C:\Users\ITO\AppData\Local\Temp\ipykernel_31224\4163572645.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  chicago_tracts['centroid'] = chicago_tracts.geometry.centroid
C:\Users\ITO\AppData\Local\Temp\ipykernel_31224\4163572645.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  chicago_tracts['latitude'] = chicago_tracts.centroid.y
C:\Users\ITO\AppData\Local\Temp\ipykernel_31224\4163572645.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  chicago_tracts['longitude'] = chicago_tracts.centroid.x


,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,perimeter,data_admin,tract_crea,date_tract,time_tract,shape_area,shape_len,centroid,latitude,longitude
0,17,031,510300,17031510300,5103,Census Tract 5103,G5020,S,2958348,0,...,0.0,0.0,None,NaT,None,1.122820e+07,14047.050709,POINT (-87.57498 41.71574),41.715737,-87.574977
1,17,031,520100,17031520100,5201,Census Tract 5201,G5020,S,2581898,521681,...,0.0,0.0,None,NaT,None,1.122820e+07,14047.050709,POINT (-87.53049 41.7203),41.720299,-87.530486
5,17,031,590700,17031590700,5907,Census Tract 5907,G5020,S,869830,12614,...,0.0,0.0,None,NaT,None,1.122820e+07,14047.050709,POINT (-87.66376 41.82681),41.826807,-87.663764
6,17,031,600400,17031600400,6004,Census Tract 6004,G5020,S,327229,0,...,0.0,0.0,None,NaT,None,1.122820e+07,14047.050709,POINT (-87.64393 41.84172),41.841718,-87.643932
13,17,031,813801,17031813801,8138.01,Census Tract 8138.01,G5020,S,841626,0,...,0.0,0.0,None,NaT,None,1.122820e+07,14047.050709,POINT (-87.74346 41.85035),41.850345,-87.743459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3207,17,031,560400,17031560400,5604,Census Tract 5604,G5020,S,339633,0,...,0.0,0.0,None,NaT,None,1.122820e+07,14047.050709,POINT (-87.74061 41.80401),41.804006,-87.740614
3208,17,031,710700,17031710700,7107,Census Tract 7107,G5020,S,490083,0,...,0.0,0.0,None,NaT,None,1.122820e+07,14047.050709,POINT (-87.65688 41.74711),41.747110,-87.656884
3211,17,031,550100,17031550100,5501,Census Tract 5501,G5020,S,7189486,1132186,...,0.0,0.0,None,NaT,None,1.122820e+07,14047.050709,POINT (-87.53533 41.66506),41.665065,-87.535333
3236,17,031,283200,17031283200,2832,Census Tract 2832,G5020,S,236547,0,...,0.0,0.0,None,NaT,None,1.122820e+07,14047.050709,POINT (-87.6592 41.86958),41.869579,-87.659202


### Testing the API and CSV formulation - single tract

In [ ]:
## API LINK - example ##
lat = 20
lon = 40
api = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=cloud_cover,visibility,relative_humidity_2m,precipitation,wind_speed_180m,wind_direction_180m,temperature_180m,surface_pressure&timezone=America%2FChicago&wind_speed_unit=mph&temperature_unit=fahrenheit&precipitation_unit=inch"

In [66]:
test_lat = 41.715737
test_lon = -87.574977
tract_id = 17031510300

url = "https://api.open-meteo.com/v1/forecast"

api_params = {
    "latitude": test_lat,
    "longitude": test_lon,
    "hourly": "cloud_cover,visibility,relative_humidity_2m,precipitation,wind_speed_180m,wind_direction_180m,temperature_180m,surface_pressure",
    "timezone": "America/Chicago",
    "wind_speed_unit": "mph",
    "temperature_unit": "fahrenheit",
    "precipitation_unit": "inch"
}
weather_data = requests.get(url, params=api_params).json()

hourly_data = weather_data["hourly"]
df = pd.DataFrame(hourly_data)
df["tract_id"] = tract_id
df

,time,cloud_cover,visibility,relative_humidity_2m,precipitation,wind_speed_180m,wind_direction_180m,temperature_180m,surface_pressure,tract_id
0,2025-07-09T00:00,99,44291.340,91,0.000,14.0,238,70.9,995.4,17031510300
1,2025-07-09T01:00,52,45931.758,91,0.000,11.5,238,71.0,994.8,17031510300
2,2025-07-09T02:00,51,44947.508,91,0.000,11.0,241,70.3,994.6,17031510300
3,2025-07-09T03:00,6,43635.172,92,0.000,10.5,254,69.6,994.5,17031510300
4,2025-07-09T04:00,50,44947.508,91,0.000,9.7,261,68.9,994.1,17031510300
...,...,...,...,...,...,...,...,...,...,...
163,2025-07-15T19:00,100,79199.477,67,0.000,15.7,213,76.6,992.9,17031510300
164,2025-07-15T20:00,100,79199.477,74,0.004,19.0,212,76.6,993.0,17031510300
165,2025-07-15T21:00,100,79199.477,84,0.004,23.3,212,76.4,993.2,17031510300
166,2025-07-15T22:00,100,79199.477,91,0.004,26.2,213,75.9,993.3,17031510300


The following cell will take about 11 minutes to run. The final product should be a csv file with 145824 rows containing all Chicago census tract weekly forecast weather data. 

In [8]:
## script to get weekly forecast weather data for each tract ##
final_csv_data = []

for idx, row in chicago_tracts.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    tract_id = row["GEOID"]
    api_params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "cloud_cover,visibility,relative_humidity_2m,precipitation,wind_speed_180m,wind_direction_180m,temperature_180m,surface_pressure",
        "timezone": "America/Chicago",
        "wind_speed_unit": "mph",
        "temperature_unit": "fahrenheit",
        "precipitation_unit": "inch"
    }

    api = "https://api.open-meteo.com/v1/forecast"    
    response = requests.get(api, params=api_params).json()

    hourly_data = response["hourly"]

    ## this creates a list that assigns tract id for each hourly date
    hourly_data["tract_id"] = [tract_id] * len(hourly_data["time"])

    ## this is appending ONE df of weather forecast for one tract 
    tract_df = pd.DataFrame(hourly_data)
    final_csv_data.append(tract_df)

    time.sleep(0.2) # to space out requests

# now we combine ALL the tract dataframes
all_tracts_csv = pd.concat(final_csv_data, ignore_index=True)
all_tracts_csv

,time,cloud_cover,visibility,relative_humidity_2m,precipitation,wind_speed_180m,wind_direction_180m,temperature_180m,surface_pressure,tract_id
0,2025-07-09T00:00,99,44291.340,91,0.000,14.0,238,70.9,995.4,17031510300
1,2025-07-09T01:00,52,45931.758,91,0.000,11.5,238,71.0,994.8,17031510300
2,2025-07-09T02:00,51,44947.508,91,0.000,11.0,241,70.3,994.6,17031510300
3,2025-07-09T03:00,6,43635.172,92,0.000,10.5,254,69.6,994.5,17031510300
4,2025-07-09T04:00,50,44947.508,91,0.000,9.7,261,68.9,994.1,17031510300
...,...,...,...,...,...,...,...,...,...,...
145819,2025-07-15T19:00,100,79199.477,86,0.008,14.3,209,77.0,990.1,17031100500
145820,2025-07-15T20:00,100,79199.477,89,0.000,18.2,208,77.0,990.2,17031100500
145821,2025-07-15T21:00,100,79199.477,91,0.000,23.2,208,76.8,990.6,17031100500
145822,2025-07-15T22:00,100,79199.477,93,0.000,26.3,209,76.2,990.7,17031100500


In [9]:
# after running successfully run this to save the csv
all_tracts_csv.to_csv("chicago_tracts_weekly_weather_forecast_07_09_2025.csv", index=False)